In [ ]:
import os 
import numpy as np
print(len(os.listdir('/content/Kiti-Single')))
testing_image =  np.load("/content/Kiti-Single/6128.npy")
print(testing_image)

17872
[[[ 24  19  16]
  [ 16  14  10]
  [ 15  13  12]
  ...
  [255 207 129]
  [255 208 126]
  [255 205 123]]

 [[ 30  20  20]
  [ 25  19  21]
  [ 35  23  24]
  ...
  [255 209 129]
  [255 211 124]
  [255 208 125]]

 [[ 39  21  19]
  [ 29  30  23]
  [ 21  29  25]
  ...
  [255 213 130]
  [255 210 128]
  [255 211 129]]

 ...

 [[ 18  16  15]
  [ 14  15  14]
  [ 15  15  19]
  ...
  [202 178 168]
  [175 157 131]
  [190 180 151]]

 [[ 15  15  13]
  [ 15  15  13]
  [ 16  16  14]
  ...
  [255 238 224]
  [184 158 134]
  [195 189 155]]

 [[  8   8   7]
  [  9   8   6]
  [  9   8   7]
  ...
  [128 128 128]
  [101  83  68]
  [ 95  99  80]]]


In [ ]:
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, Concatenate, LeakyReLU, Input, Activation, ReLU
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
weight_initialzer = RandomNormal(stddev=0.02)

img_width = 256
img_height = 256
num_channels = 3
num_stacked_imgs = 4

def residual_block(filters,previous, input):

    for i in range(9):
        x = Conv2D(filters,3,strides=1, padding='same', kernel_initializer=weight_initialzer)(input)
        x = InstanceNormalization(axis=-1)(x)
        x = ReLU()(x)
        x = Conv2D(filters,3,strides=1, padding='same', kernel_initializer=weight_initialzer)(x)
        x = InstanceNormalization(axis=-1)(x)

        if previous is not None:
            x_out = Concatenate()([previous, x])
            previous = x_out
            input = x_out

        else:
            previous = x
            input = x

    return x_out

def generator():

    x1 = Input(shape = (img_width,img_height,num_channels*num_stacked_imgs))

    x2 = Conv2D(128,kernel_size=7,strides=1,padding='same',kernel_initializer=weight_initialzer)(x1)

    x3 = Conv2D(128,kernel_size=3,strides=2,padding='same',kernel_initializer=weight_initialzer)(x2)
    x3 = InstanceNormalization(axis=-1)(x3)
    x3 = ReLU()(x3)

    x4 = Conv2D(256,kernel_size=3,strides=2,padding='same',kernel_initializer=weight_initialzer)(x3)
    x4 = InstanceNormalization(axis=-1)(x4)
    x4 = ReLU()(x4)

    x13 = residual_block(256,None,x4)

    x14 = Conv2DTranspose(128,kernel_size=3,strides=2, padding='same', kernel_initializer=weight_initialzer)(x13)
    x14 = InstanceNormalization(axis=-1)(x14)
    x14 = ReLU()(x14)

    x15 = Conv2DTranspose(256,kernel_size=3,strides=2,padding='same',kernel_initializer=weight_initialzer)(x14)
    x15 = InstanceNormalization(axis=-1)(x15)
    x15 = ReLU()(x15)

    x16 = Conv2D(3,kernel_size=7,strides=1, padding='same',kernel_initializer=weight_initialzer,activation='tanh')(x15)

    return Model(x1,x16)

def discriminator_block(filters,strides, input):

    x = Conv2D(filters,kernel_size=4, strides=strides,padding='same',kernel_initializer=weight_initialzer)(input)
    x= InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(0.2)(x)

    return x



def discriminator(num_input_images):

    x1 = Input(shape=(img_width,img_height,num_channels*num_input_images))

    x2 = Conv2D(64,kernel_size=4,strides=2,padding='same',kernel_initializer=weight_initialzer)(x1)
    x2 = LeakyReLU(0.2)(x2)

    x3 = discriminator_block(128,2,x2)

    x4 = discriminator_block(256,2,x3)

    x5 = discriminator_block(512,1,x4)

    x6 = Conv2D(1,kernel_size=4,strides=1,padding='same',kernel_initializer=weight_initialzer)(x5)

    return Model(x1,x6)

future_generator = generator()
future_generator.summary()

frame_discriminator = discriminator(1)
frame_discriminator.summary()

sequence_discriminator = discriminator(5)
sequence_discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 12 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 128 75392       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 128 147584      conv2d[0][0]                     
__________________________________________________________________________________________________
instance_normalization (Instanc (None, 128, 128, 128 256         conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
import cv2
import time
import math

lambda1 = 0.005
lambda2 = 0.003
lambda3 = 0.003

epochs = 1000

beta1 = 0.5
beta2 = 0.999

future_generator_optimizer = Adam(lr=0.0003,beta_1=beta1,beta_2= beta2)
frame_discriminator_optimizer = sequence_discriminator_optimizer = Adam(lr=0.0003,beta_1=beta1,beta_2= beta2)
loss = BinaryCrossentropy(from_logits=True)

checkpoint_dirs = '/content/drive/MyDrive/Future_Model_Bhuvan'
checkpoint_prefix = os.path.join(checkpoint_dirs,"ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = future_generator_optimizer, frame_discriminator_optimizer = frame_discriminator_optimizer, sequence_discriminator_optimizer = sequence_discriminator_optimizer, generator = future_generator, frame_discriminator = frame_discriminator, sequence_discriminator = sequence_discriminator)

def LoG(image, filtersize=7, sigma=1):

    image = (image+1)*127.5
    image = tf.squeeze(image,axis=0)
    image = tf.image.convert_image_dtype(image, tf.dtypes.float32)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, tf.dtypes.float32)
    image = tf.squeeze(image,2) 
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    n_channels = 1
    image = tf.expand_dims(image, 2)
    
    w = math.ceil(sigma * filtersize)
    w_range = int(math.floor(w/2))

    y = x = tf.range(-w_range, w_range+1, 1)
    Y, X = tf.meshgrid(x, y)
    z = tf.cast(tf.add(tf.square(X), tf.square(Y)),tf.float32)
    nom = tf.subtract(z, 2*(sigma**2))
    denom = 2*math.pi*(sigma**6)
    exp = tf.exp( -z/2*(sigma**2))
    fil = tf.divide(tf.multiply(nom, exp), denom)
    
    fil = tf.stack([fil]*n_channels, axis=2)
    fil = tf.expand_dims(fil, 3)
    
    new = tf.image.convert_image_dtype(image, tf.dtypes.float32)
    new = tf.expand_dims(new, 0)
    res = tf.nn.depthwise_conv2d(new, fil, strides=[1, 1, 1, 1], padding="SAME")
    res = tf.squeeze(res,0)
    res = tf.squeeze(res,2)
    minM = tf.math.reduce_min(res)
    maxM = tf.math.reduce_max(res)
    output = (res - minM) * 255 / (maxM - minM)
    output = tf.expand_dims(output,axis=0)
    output = output/127.5-1
    return output

def discriminator_loss(fake,real):

    return (loss(tf.ones_like(real),real) + loss(tf.zeros_like(fake),fake))*0.5

def image_similarity(img1,img2):

    return tf.reduce_mean(tf.abs(img1 - img2))

def LoG_image_similarity(img1,img2):

    return tf.reduce_mean(tf.abs(LoG(img1) - LoG(img2)))


@tf.function
def step(xm_to_xnplus1):


    xm_to_xn = xm_to_xnplus1[:,:,0:12]
    xm_to_xn = xm_to_xn/255
    xm_to_xn = tf.expand_dims(xm_to_xn,axis = 0)

    xn_plus_1 = xm_to_xnplus1[:,:,12:15]

    xn_plus_1 = xn_plus_1/127.5 - 1
    xn_plus_1 = tf.expand_dims(xn_plus_1,axis=0)

    xm_plus_1_to_xn_plus_1 = xm_to_xnplus1[:,:,3:]
    xm_plus_1_to_xn_plus_1 = xm_plus_1_to_xn_plus_1/255
    xm_plus_1_to_xn_plus_1 = tf.expand_dims(xm_plus_1_to_xn_plus_1,axis=0)
    
    xm = xm_to_xnplus1[:,:,0:3]
    xm = xm/127.5 -1
    xm = tf.expand_dims(xm,axis=0)

    with tf.GradientTape(persistent=True) as tape:

        xn_plus_1_dash = future_generator(xm_to_xn,training = True)
        xm_dash = future_generator(xm_plus_1_to_xn_plus_1[::-1],training = True)

        xm_to_xn_false = tf.concat((xm_dash,xm_to_xn[:,:,:,3:12]),axis = -1) 

        xm_plus_1_to_xn_plus_1_false = tf.concat((xm_plus_1_to_xn_plus_1[:,:,:,0:9],xn_plus_1_dash),axis=-1)

        xn_plus_1_double_dash = future_generator(xm_to_xn_false, training = True)
        xm_double_dash = future_generator(xm_plus_1_to_xn_plus_1_false[::-1])

        frame_discriminator_loss = lambda2*(discriminator_loss(frame_discriminator(xn_plus_1_dash,training=True),frame_discriminator(xn_plus_1,training=True)) + discriminator_loss(frame_discriminator(xn_plus_1_double_dash,training =True),frame_discriminator(xn_plus_1,training=True)) + discriminator_loss(frame_discriminator(xm_dash,training=True),frame_discriminator(xm,training=True)) + discriminator_loss(frame_discriminator(xm_double_dash,training=True),frame_discriminator(xm,training=True)))

        with tape.stop_recording():

            frame_discriminator_gradients = tape.gradient(frame_discriminator_loss, frame_discriminator.trainable_variables)
            frame_discriminator_optimizer.apply_gradients(zip(frame_discriminator_gradients,frame_discriminator.trainable_variables))

        xm_to_xnplus1 = tf.expand_dims(xm_to_xnplus1,axis = 0)/127.5-1

        xm_to_xn_with_xn_plus_1_dash = tf.concat((xm_to_xnplus1[:,:,:,0:12],xn_plus_1_dash),axis = -1)

        xm_to_xn_with_xn_plus_1_double_dash = tf.concat((xm_to_xnplus1[:,:,:,0:12],xn_plus_1_double_dash),axis=-1)

        xm_to_xn_with_xm_dash = tf.concat((xm_dash,xm_to_xnplus1[:,:,:,3:]),axis =-1)

        xm_to_xn_with_xm_double_dash = tf.concat((xm_double_dash,xm_to_xnplus1[:,:,:,3:]),axis=-1)

        sequence_discriminator_loss = lambda3*(discriminator_loss(sequence_discriminator(xm_to_xn_with_xm_dash,training=True), sequence_discriminator(xm_to_xnplus1,training=True)) + discriminator_loss(sequence_discriminator(xm_to_xn_with_xn_plus_1_double_dash,training=True),sequence_discriminator(xm_to_xnplus1,training=True)) + discriminator_loss(sequence_discriminator(xm_to_xn_with_xm_dash[::-1],training=True),sequence_discriminator(xm_to_xnplus1[::-1],training=True)) + discriminator_loss(sequence_discriminator(xm_to_xn_with_xm_double_dash[::-1],training=True), sequence_discriminator(xm_to_xnplus1[::-1],training=True)))

        with tape.stop_recording():

            sequence_discriminator_gradients = tape.gradient(sequence_discriminator_loss, sequence_discriminator.trainable_variables)
            sequence_discriminator_optimizer.apply_gradients(zip(sequence_discriminator_gradients,sequence_discriminator.trainable_variables))

        img_loss = image_similarity(xm,xm_dash) + image_similarity(xm,xm_double_dash) + image_similarity(xm_dash,xm_double_dash) + image_similarity(xn_plus_1,xn_plus_1_dash) + image_similarity(xn_plus_1,xn_plus_1_double_dash) + image_similarity(xn_plus_1_dash,xn_plus_1_double_dash)
        LoG_loss = LoG_image_similarity(xm,xm_dash) + LoG_image_similarity(xm,xm_double_dash) + LoG_image_similarity(xm_dash,xm_double_dash) + LoG_image_similarity(xn_plus_1,xn_plus_1_dash) + LoG_image_similarity(xn_plus_1,xn_plus_1_double_dash) + LoG_image_similarity(xn_plus_1_dash,xn_plus_1_double_dash)

        future_generator_loss = img_loss + lambda1*LoG_loss

        with tape.stop_recording():

            future_generator_gradients = tape.gradient(future_generator_loss,future_generator.trainable_variables)
            future_generator_optimizer.apply_gradients(zip(future_generator_gradients,future_generator.trainable_variables))



In [ ]:
import random
import matplotlib.pyplot as plt
from math import log10, sqrt
from tqdm import tqdm

#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dirs))

def PSNR(expected,Predicted): 
    mse = np.mean((expected - Predicted) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return mse,psnr 

def generate_images(input_array):

  first = random.randint(1,12000)
  input1 =  np.load("/content/Kiti-Single/{}.npy".format(first))
  input2 =  np.load("/content/Kiti-Single/{}.npy".format(first + 1))
  input3 =  np.load("/content/Kiti-Single/{}.npy".format(first+2))
  input4 =  np.load("/content/Kiti-Single/{}.npy".format(first + 3))
  expected = np.load("/content/Kiti-Single/{}.npy".format(first + 4))

  expected = expected/255
  input_to = np.concatenate([input1,input2,input3,input4],axis= -1)
  input_to = tf.expand_dims(input_to,axis=0)

  out1 = future_generator(input_to)
  out = tf.squeeze(out1,axis = 0)
  out = (out + 1.0)/2
  out.numpy()

  plt.figure(figsize=(15,15))
  title = ['Ground Truth','Predicted']
  display_list = [expected,out]
  for i in range(2):

        plt.subplot(1,2,i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')

  plt.show()
  mse,psnr = PSNR(expected*255,out*255)
  print("MSE value is {}".format(mse))
  print("PNSR value is {}".format(psnr))
  ssim = tf.image.ssim(expected, out, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
  print("SSIM value is {}".format(ssim))

for epoch in range(epochs):
    print('Epochs:{}'.format(epoch))
    start = time.time()
    for _,_,files in os.walk("/content/Kiti-Single"):
      random.shuffle(files)
    #Each Batch
    for k,file in enumerate(files):
        m = int(file.split('.')[0])
        # print(m)
        if m <17860: 
          # print("Loopy")
          xm = np.load("/content/Kiti-Single/{}.npy".format(m))
          xm_plus1 = np.load("/content/Kiti-Single/{}.npy".format(m+1))
          xm_plus2 = np.load("/content/Kiti-Single/{}.npy".format(m+2))
          xn = np.load("/content/Kiti-Single/{}.npy".format(m+3))
          xn_plus1 = np.load("/content/Kiti-Single/{}.npy".format(m+4))
          xm_to_xnplus1 = np.concatenate([xm,xm_plus1,xm_plus2,xn,xn_plus1], axis = -1)
          # print("before step")
          xm_to_xnplus1 = xm_to_xnplus1.astype(np.float32)
          # print(xm_to_xnplus1.dtype)
          step(xm_to_xnplus1)
          # print("After step")
          if (k+1)%100 == 0:
              print(k)

    generate_images(xm_to_xnplus1)
    print('Time Taken: {}'.format(time.time()-start))

    checkpoint.save(file_prefix=checkpoint_prefix)

Epochs:0
99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
3799
3899
3999
4099
4199
4299
4399
4499
4599
4699
4799
4899
4999
5099
5199
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799
7899
7999
8099
8199
8299
8399
8499
8599
8699
8799
8899
8999
9099
9199
9299
9399
9499
9599
9699
9799
9899
9999
10099
10199
10299
10399
10499
10599
10699
10799
10899
10999
11099
11199
11299
11399
11499
11599
11699
11799
11899
11999
12099
12199
12299
12399
12499
12599
12699
12799
12899
12999
13099
13199
13299
13399
13499
13599
13699
13799
13899
13999
14099
14199
14299
14399
14499
14599
14699
14799
14899
14999
15099


In [ ]:
checkpoint.save(file_prefix=checkpoint_prefix)

'/content/drive/MyDrive/Future_Model/ckpt-1'

In [ ]:
import numpy as np
import tensorflow as tf

input1 =  np.load("/content/drive/MyDrive/Kiti-Single/0.npy")
input2 =  np.load("/content/drive/MyDrive/Kiti-Single/1.npy")
input3 =  np.load("/content/drive/MyDrive/Kiti-Single/2.npy")
input4 =  np.load("/content/drive/MyDrive/Kiti-Single/3.npy")

tf.keras.preprocessing.image.save_img("/content/0.png",input1)
tf.keras.preprocessing.image.save_img("/content/1.png",input2)
tf.keras.preprocessing.image.save_img("/content/2.png",input3)
tf.keras.preprocessing.image.save_img("/content/3.png",input4)